# 第五讲 卷积神经网络

## 5.1 全连接网络回顾

## 5.2 卷积神经网络

## 5.3 CNN 经典网络

### 5.3.1 LeNet
借鉴点：共享卷积核，减少网络参数。

### 5.3.2 AlexNet

借鉴点：激活函数使用 Relu，提升训练速度；Dropout 防止过拟合。

### 5.3.3 VGGNet
借鉴点：小卷积核减少参数的同时，提高识别准确率；网络结构规整，适合并行加速。


### 5.3.4 InceptionNet
借鉴点：一层内使用不同尺寸的卷积核，提升感知力（通过 padding 实现输出特征面积一致）

### 5.3.5 ResNet

借鉴点：层间残差跳连，引入前方信息，减少梯度消失，使神经网络层数变身成为可能。